In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)
# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


Toggle code

In [2]:
# all modules necessary for this nb
import os
import sys
import pickle

import numpy as np
import pylab as pl
from sklearn.covariance import EmpiricalCovariance
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.metrics import silhouette_score as clust_score
from sklearn.preprocessing import StandardScaler
from scipy import stats as sstats

# setting parameters for default matplotlib plots
%matplotlib inline

In [3]:
pl.rcParams['savefig.dpi'] = 300 # dpi for most publications
pl.rcParams['figure.dpi'] = 300 # dpi for most publications
pl.rcParams['xtick.labelsize'] = 7
pl.rcParams['ytick.labelsize'] = 7
pl.rcParams['axes.labelsize'] = 7
from ipywidgets import interact

# needs to find the library of functions
sys.path.append('../../../../../code/')  # to be replaced!

import utils as ut
import plots as pt

In [4]:
# a double percentage sign indicates a magic function. in this case, now we are writing this cell in javascript.

In [5]:
NOTEBOOK_NAME = 'preprocessing'

In [6]:
from pickleshare import PickleShareDB

autorestore_folder = os.path.join(os.getcwd(), 'autorestore', NOTEBOOK_NAME)
db = PickleShareDB(autorestore_folder)
import sys
from workspace import *
import IPython
ip = IPython.get_ipython()

# this will restore all the saved variables. ignore the errors listed.
load_workspace(ip, db)

# use `save_worspace(db)` to save variables at the end

In [7]:
data_folder = '../data'

In [8]:
traces = np.loadtxt(os.path.join(data_folder, 'C.txt')).T #denoised traces
traces_raw = np.loadtxt(os.path.join(data_folder, 'C_raw.txt')).T
try:
    areas = ut.load_spatial_footprints_A(os.path.join(data_folder, 'A.txt'))
except:
    print 'not 512 by 512'
events = np.loadtxt(os.path.join(data_folder, 'S.txt')).T
dff = np.loadtxt(os.path.join(data_folder, 'C_df.txt')).T
mean_image, contours = ut.load_spatial_footprints(os.path.join(data_folder, 'Coor.mat'),
                                                   os.path.join(data_folder, 'Cnn.txt'),
                                                   key='coor')

#filename = os.path.join(data_folder, 'behavior.txt')

# adapting above code so we don't have to rename every arudino file to 'behavior.txt' when importing
for file in os.listdir(data_folder):
    if file.endswith("codes.txt"):
        filename = os.path.join(data_folder, file)
    elif 'behavior' in file:
        filename = os.path.join(data_folder, file)
print filename

behavior = ut.read_behavior(filename)
events_list = np.unique([b[1] for b in behavior])

../data/c10m6-013020-D1_codes.txt


In [9]:
# grab time axis from the xml file

import xml.etree.ElementTree as ET
#xmlfile = os.path.join(data_folder, 'tseries.xml')

#adapting above code so we don't have to rename imported .xml file
for file in os.listdir(data_folder):
    if file.endswith(".xml"):
        xmlfile = data_folder + '/' + file
print "I infer the time axis from:\n", xmlfile
tree = ET.parse(xmlfile)
root = tree.getroot()

# unfortunately we miss the first frame
time_ax = np.r_[[child.attrib['absoluteTime']
                 for child in root.iter('Frame')]].astype(float)

I infer the time axis from:
../data/TSeries-01302020-c10m6-650um-000.xml


In [10]:
# sync times
start_2p = ut.parse_behavior(behavior, 'BEGIN')[0]
behavior = [[float(b[0])-start_2p, b[1]] for b in behavior]
time_ax -= time_ax[0]

In [11]:
# make sure presentations are correct in timing.
behavior[:10]

[[0.0, 'BEGIN'],
 [6.0, 'ODOR6'],
 [20.0, 'END'],
 [27.124000000000024, 'BEGIN'],
 [33.124000000000024, 'ODOR6'],
 [47.124000000000024, 'END'],
 [57.7650000000001, 'BEGIN'],
 [63.7650000000001, 'ODOR4'],
 [77.7650000000001, 'END'],
 [88.533000000000015, 'BEGIN']]

In [12]:
# clean up artefact events at the beginning of each cycle - will need for odors, but not sucrose and shock as
# suc and shock was continuous imaging
#for s, e in cycles:
#    if s>np.max(time_ax): break
#    events[np.where(time_ax>=s)[0][0]] = 0

In [13]:
##PLOTTING TRACES

# for i in range(traces.shape[1]):
    
#     fig, axs = pl.subplots(1, 1, figsize=(3, 2),)
#     pl.plot(time_ax, traces[:, i])
#     #pl.vlines(time_ax[np.nonzero(events[:, 0])], -2, 0, lw=1)

#     fig.savefig('../img/%s__traces_pre.pdf'%i)

In [14]:
print len(traces)
print len(time_ax)

7872
47237


In [15]:
time_ax = time_ax[::6] # use this if video was averaged and need to adjust xml output to match

In [16]:
#ratio = int(np.floor(time_ax.shape[0]/traces.shape[0]))
#print ratio

In [17]:
#time_ax = time_ax[::ratio] # use this if video was averaged and need to adjust xml output to match

In [18]:
print len(traces)
print len(time_ax)

7872
7873


In [19]:
time_ax = time_ax[0:len(traces)] # use this if any video frames were truncated (often need to do this a video is averaged)

In [20]:
print len(traces)
print len(time_ax)

7872
7872


### Only keep portion of CNMFe files that correspond to behavior file (use if CNMFe was performed on a concatenated video file)

In [21]:
#traces = traces[(7837-len(time_ax)):]
#traces_raw = traces_raw[(7837-len(time_ax)):]
#events = events[(7837-len(time_ax)):]
#dff = dff[(7837-len(time_ax)):]

In [22]:
#to correct for different clocking speeds between arduino and 2p software
behavior = ut.sync_behavior_to_xml(time_ax, behavior)

In [23]:
# -----------------------------------------------------------
# these times are relative to the single cycle
# and centered around tone onset
CONTINUOUS = True
CYCLE_START = -6  # seconds
US_DURATION = 2  # seconds  // IS THIS FIXED?
ANALYSIS_WINDOW = 2  # seconds. How long of time window do we want to analyze over?
AFTER_US_PERIOD = 4
REWARD_WIN = 2
CYCLE_DURATION = abs(CYCLE_START) + US_DURATION + AFTER_US_PERIOD
US_START = 0
US_END = US_START + US_DURATION

# -----------------------------------------------------------
# these times are absolute times, taken from the arduino file
# when the tones starts and ends
rewards = np.r_[ut.parse_behavior(behavior, 'REWARD')]
shocks = np.r_[ut.parse_behavior(behavior, 'SHOCK')]
coyote = np.r_[ut.parse_behavior(behavior, 'ODOR6')]
female = np.r_[ut.parse_behavior(behavior, 'ODOR4')]
blasts = np.r_[ut.parse_behavior(behavior, 'blast')]
collected = np.r_[ut.parse_behavior(behavior, 'COLLECTED')]
licks = np.r_[ut.parse_behavior(behavior, 'LICK')]


# -----------------------------------------------------------
# when the experiment starts and ends, in absolute time
# begin_end = ut.parse_behavior(behavior, '[be]')
# when each cycle starts and ends
# (last cycle is usually oddly recorded)
if CONTINUOUS:
    cycles_starts = ut.parse_behavior(behavior, ('^[ORSb]'), offset=CYCLE_START) #looks for arduino line that begins w/ either O, R, S, or b
    cycles_ends = ut.parse_behavior(behavior, ('^[ORSb]'), offset=CYCLE_DURATION+CYCLE_START)
else:
    cycles_starts = ut.parse_behavior(behavior, 'BEGIN')
    cycles_ends = ut.parse_behavior(behavior, 'END')
cycle_subtract = 0   #do we need to subtract off the last cycle because it's too short???
if cycle_subtract !=0:
    cycles = np.r_[zip(cycles_starts,  # offset will be ADDED, with sign
                   cycles_ends)][:cycle_subtract]
else:
    cycles = np.r_[zip(cycles_starts,  # offset will be ADDED, with sign
                   cycles_ends)]
print 'we are subtracting off this many cycles'
print cycle_subtract
# -----------------------------------------------------------
# which trials are a.p. and which reward
is_rewardt = [any(map(lambda t: (t>=s) and (t<e), rewards)) for s, e in zip(cycles_starts, cycles_ends)]
is_shockt = [any(map(lambda t: (t>=s) and (t<e), shocks)) for s, e in zip(cycles_starts, cycles_ends)]
is_femalet = [any(map(lambda t: (t>=s) and (t<e), female)) for s, e in zip(cycles_starts, cycles_ends)]
is_coyotet = [any(map(lambda t: (t>=s) and (t<e), coyote)) for s, e in zip(cycles_starts, cycles_ends)]
is_blastt = [any(map(lambda t: (t>=s) and (t<e), blasts)) for s, e in zip(cycles_starts, cycles_ends)]

we are subtracting off this many cycles
0


In [24]:
reward_times = []
for s, e in cycles[is_rewardt]:
# s, e = cycles[np.where(is_rewardt)[0][i]]
    try:
        r = rewards[(rewards>=s)*(rewards<e)][0]
        later_licks = licks-r
        reward_times.append(later_licks[(later_licks>=0)][0])
    except:
        reward_times.append(np.nan)

In [25]:
reward_times = np.r_[reward_times]

In [26]:
print reward_times

[  1.34400235e+01   5.37153294e+00   6.84355652e-01   1.47160776e+00
   1.20799429e+00   2.92287193e+00   5.71161016e+00   3.13113085e+00
   2.09967996e+00   4.24446796e-01   9.89719058e-03   1.39975048e-01
   2.48124658e+00   4.63418452e-01   4.38838226e+00   3.00156865e+00
   7.40477359e-01   4.01897304e-01   3.85078874e+00   3.24397009e+00
   1.66033939e+00   9.61199063e-01   1.98324304e+00   8.79994890e-02
   1.08595255e+00   2.58887043e+00   1.89140274e+00   1.39124083e+00
   3.26163840e+00   5.88884187e-02]


In [27]:
is_rewarded = reward_times<12

In [28]:
is_rewarded

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True], dtype=bool)

In [29]:
time_ax_single = ut.extract_single_cycle_time_ax(time_ax, cycles,
                                                 cycle_duration=CYCLE_DURATION, cycle_start=CYCLE_START)

In [30]:
save_workspace(db)

Could not store variable 'IPython'. Skipping...
Could not store variable 'pt'. Skipping...
Could not store variable 'sstats'. Skipping...
Could not store variable 'pl'. Skipping...
Could not store variable 'di'. Skipping...
Could not store variable 'pickle'. Skipping...
Could not store variable 'ut'. Skipping...
Could not store variable 'ip'. Skipping...
Could not store variable 'np'. Skipping...
Could not store variable 'sys'. Skipping...
Could not store variable 'ET'. Skipping...
Could not store variable 'os'. Skipping...


### 